In [5]:
from dotenv import load_dotenv
import os
import time
import json
import pymongo

from openai import AzureOpenAI
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
client = pymongo.MongoClient(CONNECTION_STRING)
# Create database to hold cosmic works data
# MongoDB will create the database if it does not exist
db = client.cms_open


In [3]:
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-3-small"
COMPLETIONS_DEPLOYMENT_NAME = "gpt-4"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2024-02-01"

In [4]:
ai_client = AzureOpenAI(
    azure_endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    api_key = AOAI_KEY
    )

In [6]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_embeddings(text: str):
    '''
    Generate embeddings from string of text using the deployed Azure OpenAI API embeddings model.
    This will be used to vectorize document data and incoming user messages for a similarity search with
    the vector index.
    '''
    response = ai_client.embeddings.create(input=text, model=EMBEDDINGS_DEPLOYMENT_NAME)
    embeddings = response.data[0].embedding
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI
    return embeddings

In [7]:
def vector_search(collection_name, query, num_results=3):
    """
    Perform a vector search on the specified collection by vectorizing
    the query and searching the vector index for the most similar documents.

    returns a list of the top num_results most similar documents
    """
    collection = db[collection_name]
    query_embedding = generate_embeddings(query)    
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

def print_chunk_search_result(result):
    '''
    Print the search result document in a readable format
    '''
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"_id: {result['document']['_id']}\n")

In [8]:
system_prompt = """
You are a helpful, fun and friendly healthcare policy expert, with a focus on publications from the CMS. 

You are designed to answer questions about how rule changes will impact users from various roles in the healthcare industry.

Answer questions related to the information provided in the list of text chunks below that are represented
in JSON format.

"""

In [9]:
def rag_with_vector_search(question: str, num_results: int = 3):
    """
    Use the RAG model to generate a prompt using vector search results based on the
    incoming question.  
    """
    # perform the vector search and build document chunk list
    results = vector_search("cms_open", question, num_results=num_results)
    chunk_list = ""
    for result in results:
        if "contentVector" in result["document"]:
            del result["document"]["contentVector"]
        chunk_list += json.dumps(result["document"], indent=4, default=str) + "\n\n"

    # generate prompt for the LLM with vector results
    formatted_prompt = system_prompt + chunk_list

    # prepare the LLM request
    messages = [
        {"role": "system", "content": formatted_prompt},
        {"role": "user", "content": question}
    ]

    completion = ai_client.chat.completions.create(messages=messages, model=COMPLETIONS_DEPLOYMENT_NAME)
    return completion.choices[0].message.content

In [10]:
print(rag_with_vector_search("What rule changes are going to affect me, a doctor in rural Colorado that has a large number of homeless patients.", 5))

The rules highlighted in the texts can significantly affect your practice:

1. Change to Z Codes Severity for Homelessness: The severity level designation for the social determinants of health (SDOH) diagnosis codes describing homelessness will be changed from non-complication or comorbidity (NonCC) to complication or comorbidity (CC) starting FY 2024 (Text ID: c59f8b9c-64a0-4432-8b8a-367a58b995cc). This change recognizes homelessness as an indicator of increased resource utilization in the acute inpatient hospital setting, which could potentially result in increased funding or resources for care of homeless patients.

2. Rule Changes for Rural Hospitals: For rural hospitals, changes are coming through the TEAM model (Text ID: f59f7097-51c5-4658-939d-42ce23736287). This may influence how your practice is funded or operates. Additionally, "Rural Extension Hospitals (REH)" status will be available for facilities that meet certain specifications (Text ID: 174a5f84-a17c-4889-8239-d476acd46